<a href="https://colab.research.google.com/github/upadhyayprakash/eva4/blob/master/S4/EVA4_Session_4_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Task
Train MNIST to **99.4%** or more validation accuracy with following constraints:
1. Less than 20k parameters
2. less than 20 Epochs
3. No Fully Connected Layer(almost)
4. Try all the concepts taught in the class

### RESULT
1. Val Accuracy: **94%** at 17th Epoch
2. No. of parameters: 15,722
3. No. of Epoch: 17th Epoch (for 99.4% Accuracy)
4. Without any Fully Connected Layer
5. Concepts Used: 1x1 Conv, Batch Normalization, Dropout, Avg Pooling, ReLU, Batch_Size, Optimizer, transforms module for Image Augmentation.

NOTE: Tried using CutOut, work in Progress.

In [0]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
# Old Architecture, Don't use this
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        print('#### OLD ####')
        print(x.shape)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
# Returns a Cut-Out image for training data
import numpy as np

def cutout(mask_size, p, cutout_inside, mask_color=(0, 0, 0)):
    mask_size_half = mask_size // 2
    offset = 1 if mask_size % 2 == 0 else 0

    def _cutout(image):
        image = np.asarray(image).copy()

        if np.random.random() > p:
            return image

        h, w = image.shape[:2]

        if cutout_inside:
            # print('Yess')
            cxmin, cxmax = mask_size_half, w + offset - mask_size_half
            cymin, cymax = mask_size_half, h + offset - mask_size_half
        else:
            # print('Nooo')
            cxmin, cxmax = 0, w + offset
            cymin, cymax = 0, h + offset

        cx = np.random.randint(cxmin, cxmax)
        cy = np.random.randint(cymin, cymax)
        xmin = cx - mask_size_half
        ymin = cy - mask_size_half
        xmax = xmin + mask_size
        ymax = ymin + mask_size
        xmin = max(0, xmin)
        ymin = max(0, ymin)
        xmax = min(w, xmax)
        ymax = min(h, ymax)
        image[ymin:ymax, xmin:xmax] = mask_color
        return image

    return _cutout

In [0]:
class NewNet(nn.Module):
    def __init__(self):
        super(NewNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3) #input - 28x28x1 | Kernel Size - 3x3x1 | Output - 28x28x16 | RF = 3
        self.conv2 = nn.Conv2d(16, 16, 3)
        self.conv3 = nn.Conv2d(16, 16, 3)
        self.conv4 = nn.Conv2d(16, 16, 3)
        self.conv5 = nn.Conv2d(16, 16, 3)
        self.conv6 = nn.Conv2d(16, 16, 3)
        self.conv7 = nn.Conv2d(16, 16, 3)
        self.batchNorm1 = nn.BatchNorm2d(16)
        self.batchNorm2 = nn.BatchNorm2d(16)
        self.batchNorm3 = nn.BatchNorm2d(16)
        self.batchNorm4 = nn.BatchNorm2d(16)
        self.batchNorm5 = nn.BatchNorm2d(16)
        self.batchNorm6 = nn.BatchNorm2d(16)
        self.batchNorm7 = nn.BatchNorm2d(16)
        self.conv7FC = nn.Conv2d(16, 10, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.convOne1 = nn.Conv2d(32, 16, 1)
        self.avgPool1 = nn.AvgPool2d(2)
        self.avgPool2 = nn.AvgPool2d(2)

        self.drop1 = nn.Dropout2d(0.20)
        self.drop2 = nn.Dropout2d(0.20)
        self.drop3 = nn.Dropout2d(0.20)
        self.drop4 = nn.Dropout2d(0.20)
        self.drop5 = nn.Dropout2d(0.20)
        self.drop6 = nn.Dropout2d(0.20)
        
    def forward(self, x):
        x = self.batchNorm1(F.relu(self.conv1(x)))
        x = self.drop1(x)
        # print(x.shape)
        
        x = self.batchNorm3(F.relu(self.conv2(x)))
        self.drop2 = nn.Dropout2d(0.20)
        # print(x.shape)
        
        x = self.batchNorm4(F.relu(self.conv3(x)))
        self.drop3 = nn.Dropout2d(0.20)
        # print(x.shape)
        
        x = self.avgPool1(x)
        # print(x.shape)

        x = self.batchNorm5(F.relu(self.conv4(x)))
        self.drop4 = nn.Dropout2d(0.20)
        # print(x.shape)
        
        x = self.batchNorm6(F.relu(self.conv5(x)))
        self.drop5 = nn.Dropout2d(0.20)
        # print(x.shape)

        x = self.batchNorm7(F.relu(self.conv6(x)))
        # print(x.shape)

        x = F.relu(self.conv7(x))
        # print(x.shape)

        x = self.conv7FC(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [5]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = NewNet().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
         Dropout2d-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
            Conv2d-6           [-1, 16, 22, 22]           2,320
       BatchNorm2d-7           [-1, 16, 22, 22]              32
         AvgPool2d-8           [-1, 16, 11, 11]               0
            Conv2d-9             [-1, 16, 9, 9]           2,320
      BatchNorm2d-10             [-1, 16, 9, 9]              32
           Conv2d-11             [-1, 16, 7, 7]           2,320
      BatchNorm2d-12             [-1, 16, 7, 7]              32
           Conv2d-13             [-1, 16, 5, 5]           2,320
      BatchNorm2d-14             [-1, 1

<ipython-input-4-984d5a73ab85>:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [0]:

torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/home/prakash/Prakash/EVA4/Session-4/Notebooks/dataMNIST', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)),
                        # transforms.ToPILImage(),
                        # transforms.RandomRotation(20),
                        # cutout(2,
                        #     0.27,
                        #     False
                        #   )
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/home/prakash/Prakash/EVA4/Session-4/Notebooks/dataMNIST', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
# Loading the model to GPU
print('Loading to GPU')
model = NewNet().to(device)
print('Loaded to GPU')
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) #Stochastic Gradient Descent Optimizer
print('Done')
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]

Loading to GPU
Loaded to GPU
Done


<ipython-input-4-984d5a73ab85>:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0509, Accuracy: 9833/10000 (98.33%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0225, Accuracy: 9940/10000 (99.40%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0284, Accuracy: 9921/10000 (99.21%)



loss=8.934736251831055e-05 batch_id=937: 100%|██████████| 938/938 [00:10<00:00, 87.22it/s]



Test set: Average loss: 0.0236, Accuracy: 9931/10000 (99.31%)



In [9]:
import sys
print(sys.version_info)


sys.version_info(major=3, minor=8, micro=1, releaselevel='final', serial=0)
